In [1]:
import pandas as pd

In [12]:
df = pd.read_excel("/media/cattiaux/DATA/Wassati/team_data/schneider/OEM_Verbatims_base.xlsx", engine='openpyxl')
df_labelled = pd.read_csv("/media/cattiaux/DATA/Wassati/team_data/schneider/df_all_labelled.csv", dtype={'year': str})

/tmp/ipykernel_15528/3454085751.py:2: DtypeWarning: Columns (10,11,15,19,20,21,23,34,35,42,43,44,45) have mixed types. Specify dtype option on import or set low_memory=False.
  df_labelled = pd.read_csv("/media/cattiaux/DATA/Wassati/team_data/schneider/df_all_labelled.csv", dtype={'year': str})


In [13]:
df2 = pd.read_csv("/media/cattiaux/DATA/Wassati/team_data/schneider/OEM_Verbatims_allComment.csv")

In [11]:
len(df)

55372